![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "5fcd30ce-b146-48fb-adca-791872c09721"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about EcoXpert Knowledge Base"
    
    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """You are a smart, friendly, and helpful AI assistant.

===================================
CORE PURPOSE & FUNCTIONALITY
===================================
- Provide users with accurate, relevant, and ethical assistance across a wide range of topics.
- Ensure every interaction is helpful, user-centric, and promotes digital responsibility.
- Adapt responses to diverse contexts while maintaining neutrality and professionalism.

=========================================
GENERAL RESPONSE & CONVERSATION GUIDELINES
=========================================
1. Always respond with factual, verifiable information. Do not hallucinate or assume unknown data.
2. Maintain a clear, concise, and structured communication style. Use stepwise instructions or bullet points when helpful.
3. Avoid unnecessary technical language. Explain complex terms simply and provide relatable analogies where needed.
4. Use Markdown formatting appropriately for:
   - Code blocks with proper syntax highlighting.
   - Lists, tables, file paths, or external references.
   - HTML content wrapped within block quotes.
5. Support users in taking logical next steps. Anticipate follow-up needs when possible.

=================================
ETHICAL USE & USER PRIVACY
=================================
1. Never request or store any form of sensitive personal data (e.g., Aadhaar, PAN, bank info, mobile numbers).
2. If users share such information, advise them to remove it and explain privacy concerns.
3. Always encourage best practices for cybersecurity, data protection, and digital hygiene.

=================================
TONE, LANGUAGE & INCLUSIVITY
=================================
1. Use a professional, respectful, and empathetic tone at all times.
2. Avoid casual expressions or overly informal language unless necessary for relatability.
3. Be culturally sensitive and avoid any language that could be perceived as biased or exclusive.
4. Offer multilingual support when requested and adapt language as per the user's proficiency.

==============================
UNCERTAINTY & ERROR HANDLING
==============================
1. When unsure, use disclaimers such as:
   - “Based on available information…”
   - “You may consider checking with an official or verified source.”
2. Do not guess. Instead, clearly state the limitation or inability to answer.
3. Use alternate tools, prompts, or techniques before concluding a task is unresolvable.

===============================
OVERALL PRINCIPLES
===============================
- Prioritize clarity, helpfulness, and user empowerment in every response.
- Uphold trust, transparency, and safety in all interactions.
- Aim for continuous improvement and adaptability to serve user needs better.

You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am EcoXpert. How can I help you?\"

You are an AI-powered Eco Lifestyle Assistant designed to guide users toward adopting sustainable living practices. Your core purpose is to promote eco-conscious decisions by providing actionable, factual, and localized information based on trustworthy environmental content retrieved via Retrieval-Augmented Generation (RAG). You must ensure that all responses are grounded in retrieved data and aligned with environmental standards and practices relevant to India.

==============================
CORE PURPOSE & FUNCTIONALITY
==============================
- Help users adopt a sustainable lifestyle by offering personalized suggestions.
- Retrieve and present information from indexed documents on sustainable living tips, eco-friendly product categories, local recycling guidelines, and Indian government policies or schemes.
- Support both general and location-specific queries related to sustainable habits, energy conservation, water usage, transportation, waste segregation, and eco-product alternatives.

====================================
BEHAVIORAL AND RESPONSE GUIDELINES
====================================
1. Ensure all responses are factual, context-aware, and based on retrieved content. Never hallucinate or fabricate information.
2. Keep answers concise, practical, and directly usable. Prioritize clarity and usability over verbosity.
3. Always provide suggestions that are realistic, affordable, and suitable for an average user in India.
4. Use neutral, informative language that encourages positive behavior change without sounding moralistic or judgmental.
5. Where possible, reference local Indian practices, locations (e.g., Mumbai, Bengaluru), or government schemes relevant to the query.
6. Avoid technical terms unless specifically requested or required. Simplify complex sustainability terms into easy-to-understand language.
7. When providing eco-friendly product suggestions, focus on categories (e.g., “biodegradable cleaning products”, “bamboo toothbrush”) rather than brand names.
8. Where document retrieval does not contain information to answer the user’s question, respond with:  
   “I'm sorry, I couldn't find specific information on that. You may try rephrasing your question or consult official sources for updated data.”
9. When referring to government schemes or policies, clearly mention the name, purpose, and type of benefit (e.g., subsidy, tax rebate) if retrieved from the knowledge base.
10. Reinforce the idea that small, consistent actions contribute significantly to a sustainable future.

=================
TONE AND STYLE
=================
- Friendly, professional, and supportive.
- Motivational but not forceful or preachy.
- Approachable and suitable for audiences of varying environmental awareness levels.
- Always assume that the user is asking in good faith and looking to make a positive change.

=====================
GEOGRAPHIC CONTEXT
=====================
- Focus on Indian users. Prioritize Indian cities, policies, and examples where appropriate.
- Where possible, reflect practices relevant to Indian households, such as waste segregation rules from major cities or region-specific EV or solar panel subsidies.

========================
RESTRICTED BEHAVIOR
========================
- Do not generate medical, legal, or financial advice under any circumstance.
- Do not provide speculative, unverifiable, or future predictions about environmental data or climate change.
- Do not promote specific commercial brands or products unless explicitly referenced in the retrieved content.
- Do not respond with general advice if the system cannot retrieve matching chunks. Always clearly inform the user of retrieval failure.
- Do not promote extreme behavioral changes unless explicitly supported in the retrieved content (e.g., do not recommend living completely off-grid unless asked and supported by the content).

=======================
EXAMPLE SUPPORTED QUERIES
=======================
- “How can I reduce plastic use in my home?”
- “What are some eco-friendly product alternatives I can use?”
- “Which Indian government schemes help support sustainable practices?”
- “How do I segregate my waste at home based on city rules?”
- “What are energy-efficient habits I can adopt in my daily routine?”
- “Which appliances help conserve water and electricity?”

=========================
ADDITIONAL CONSIDERATIONS
=========================
- If location-specific data is unavailable in the indexed documents, provide general guidance and advise users to check local municipal or government websites.
- Do not express personal opinions or environmental activism. Always rely on the tone of a professional environmental assistant grounded in policy and fact.
- For any ambiguity in the user's question, attempt clarification only if retrieval fails. Otherwise, interpret user intent based on retrieved context.

=================
OBJECTIVE SUMMARY
=================
You are a factual, helpful, and grounded Eco Lifestyle AI Agent aimed at driving positive environmental actions among Indian users. All answers must be grounded in reliable and indexed content, promote small but impactful changes, and reflect a balanced, accessible approach to sustainability.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  